In [2]:
import torch   #查看torch版本
import torch.nn.functional as F
print(torch.__version__)  #注意是双下划线

import torchvision  #查看torchvision版本
print(torchvision.__version__)  #注意是双下划线

print(torch.version.cuda)  #注意是双下划线

print(torch.cuda.is_available())#如果cuda可用，则返回True

2.0.1
0.15.2
11.7
True


In [4]:
a=torch.randn(3,4)
weights = F.softmax(a, dim=1)
print(weights)

tensor([[0.3810, 0.1712, 0.0594, 0.3884],
        [0.1361, 0.4761, 0.2130, 0.1748],
        [0.0412, 0.5379, 0.2314, 0.1895]])


In [18]:
def similarity_matrix(X, sigma = 1.0, gamma = 2.0):
        dists = torch.cdist(X, X)
        sims = torch.exp(-dists / (sigma * dists.mean(dim = -1).mean(dim = -1).reshape(-1, 1, 1)))
        return sims
X=torch.randn(8,25,32)
sims = similarity_matrix(X)
print(sims.shape)

torch.Size([8, 25, 25])


In [25]:
batch_num, neighbor_num, feature_size =X.shape
nodes_selected = []
cache = torch.zeros((batch_num, 1, neighbor_num))
for i in range(20):
    gain = torch.sum(torch.maximum(sims, cache) - cache, dim = -1)
    selected = torch.argmax(gain, dim = 1)
    cache = torch.maximum(sims[torch.arange(batch_num), selected].unsqueeze(1), cache)
    nodes_selected.append(selected)
    

In [24]:
print(gain.shape)
print(selected)
print(sims[torch.arange(batch_num), selected].unsqueeze(1).shape)

torch.Size([8, 25])
tensor([21, 20, 20,  9, 14, 22, 19,  9])
torch.Size([8, 1, 25])


In [26]:
print(nodes_selected)

[tensor([21, 20, 20,  9, 14, 22, 19,  9]), tensor([14, 15,  5, 14,  9, 17,  7, 13]), tensor([23,  3, 22, 12, 22,  5, 11,  8]), tensor([ 5,  9, 15,  3, 13,  2,  8,  0]), tensor([ 2,  1, 21,  7, 24, 15, 24, 23]), tensor([19, 10,  4, 10, 23,  4, 13,  6]), tensor([22,  7,  2, 23,  4, 20, 21, 14]), tensor([ 0, 16,  7, 17, 16, 16, 12,  3]), tensor([10,  0, 11,  6,  7, 21, 16, 19]), tensor([24, 19,  1, 11, 18, 24,  9, 17]), tensor([ 8, 24, 12,  5, 11, 18, 10,  1]), tensor([18,  8,  9, 21, 12,  1, 14,  4]), tensor([13, 17, 10,  4, 21, 23,  2,  7]), tensor([ 4, 22,  8,  2, 17,  3,  1, 11]), tensor([11,  4,  3, 15, 10, 13,  3, 18]), tensor([20,  2,  0, 22,  8, 11, 22, 24]), tensor([ 1,  5, 16, 24,  2, 12,  5,  2]), tensor([ 6, 11, 17, 18,  3, 14, 20, 12]), tensor([16, 21, 23, 19, 15, 10, 23, 15]), tensor([17, 13, 14, 13,  6,  8,  6, 22])]


In [2]:
a = torch.nn.Parameter(torch.randn(4,2))
print(a)
muti_int=torch.randn(2,2,2)
relu = torch.nn.PReLU()
print(relu(a))
# print(muti_int)
# print(torch.sum(muti_int*a,dim=1))
# print((a*muti_int).sum(dim=1))

Parameter containing:
tensor([[ 0.7334,  0.1030],
        [-2.3771, -1.2392],
        [ 1.2964, -0.1500],
        [ 2.5213,  0.6338]], requires_grad=True)
tensor([[ 0.7334,  0.1030],
        [-0.5943, -0.3098],
        [ 1.2964, -0.0375],
        [ 2.5213,  0.6338]], grad_fn=<PreluKernelBackward0>)


In [3]:
import dgl
import torch as th
# 边 0->1, 0->2, 0->3, 1->3
u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
g = dgl.graph((u, v))
print(g) # 图中节点的数量是DGL通过给定的图的边列表中最大的点ID推断所得出的
# 获取节点的ID
print(g.nodes())
# 获取边的对应端点
print(g.edges())
# 获取边的对应端点和边ID
print(g.edges(form='all'))

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})
tensor([0, 1, 2, 3])
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))


In [4]:
import sys
import dgl
import dgl.function as fn
import os
import multiprocessing as mp
from tqdm import tqdm
import pdb
import numpy as np
import torch
import torch.nn as nn
import logging
from utils.parser import parse_args
from utils.dataloader import Dataloader
from utils.utils import config, construct_negative_graph, choose_model, load_mf_model, NegativeGraph
from utils.tester import Tester
from models.sampler import NegativeSampler
import wandb

In [5]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default = 'TaoBao', type = str,
                    help = 'Dataset to use')
parser.add_argument('--seed', default = 2022, type = int,
                    help = 'seed for experiment')
parser.add_argument('--embed_size', default = 32, type = int,
                    help = 'embedding size for all layer')
parser.add_argument('--lr', default = 0.05, type = float,
                    help = 'learning rate')
parser.add_argument('--weight_decay', default = 8e-8, type = float,
                    help = "weight decay for adam optimizer")
#######################################################################
parser.add_argument('--model', default ='test',type = str,
                    help = 'model selection')#dgrec base moe test
parser.add_argument('--epoch', default = 1000, type = int,
                    help = 'epoch number')
parser.add_argument('--patience', default = 10, type = int,
                    help = 'early_stop validation')
parser.add_argument('--batch_size', default = 2048, type = int,
                    help = 'batch size')
parser.add_argument('--layers', default = 1, type = int,
                    help = 'layer number')
parser.add_argument('--gpu', default = 0, type = int,
                    help = '-1 for cpu, 0 for gpu:0')
parser.add_argument('--k_list', default = [100, 300], type = list,
                    help = 'topk evaluation')
parser.add_argument('--k', default = 20, type = int,
                    help = 'neighbor number in each GNN aggregation')
parser.add_argument('--neg_number', default = 4, type = int,
                    help = 'negative sampler number for each positive pair')
parser.add_argument('--metrics', default = ['recall', 'hit_ratio', 'coverage'])


parser.add_argument('--sigma', default = 1.0, type = float,
                    help = 'sigma for gaussian kernel')
parser.add_argument('--gamma', default = 2.0, type = float,
                    help = 'gamma for gaussian kernel')
################################################################################
parser.add_argument('--category_balance', default = True, type = bool,
                    help = 'whether make loss category balance')
parser.add_argument('--beta_class', default = 0.9, type = float,
                    help = 'class re-balanced loss beta')
parser.add_argument('--context_code_dim', default = 32, type = int,
                    help = 'interest num')
parser.add_argument('--num_context_codes', default = 32, type = int,
                    help = 'interest dim')
parser.add_argument('--n_experts', default = 5, type = int,
                    help = 'n_experts')
##########################################################################################
parser.add_argument('--wandb_enable', default = True, type = bool,
                    help = 'layer number')
parser.add_argument('--hidden_size', default = 32, type = int,
                        help = 'n_experts')
parser.add_argument('--k_experts', default = 2, type = int,
                            help = 'n_experts')
parser.add_argument('--moe', default = False, type = bool,
                            help = 'layer number')
################################################################################
parser.add_argument('--sub', default = 'rand', type = str,
                            help = 'layer number')

args = parser.parse_args([])

In [6]:
data = args.dataset
if args.gpu >= 0 and torch.cuda.is_available():
        device = 'cuda:{}'.format(args.gpu)
else:
        device = 'cpu'
device = torch.device(device)
args.device = device
dataloader = Dataloader(args, data, device)
graph_pos = dataloader.train_graph

100%|██████████| 136710/136710 [00:00<00:00, 4557511.42it/s]


In [24]:
etype='rate'
numbers=graph_pos.edges(etype='rate')[0]
print(numbers)

tensor([    0,     0,     0,  ..., 82632, 82632, 82632], device='cuda:0')


In [25]:
from collections import Counter
numbers = numbers.tolist()
counts = Counter(numbers)

# 打印每个元素及其个数
sum=0
for num, count in counts.items():
    print(f"数字 {num} 的个数：{count}")
    sum+=count
print(sum/82632)


数字 0 的个数：16
数字 1 的个数：12
数字 2 的个数：16
数字 3 的个数：27
数字 4 的个数：24
数字 5 的个数：11
数字 6 的个数：27
数字 7 的个数：72
数字 8 的个数：17
数字 9 的个数：51
数字 10 的个数：42
数字 11 的个数：59
数字 12 的个数：24
数字 13 的个数：15
数字 14 的个数：27
数字 15 的个数：13
数字 16 的个数：21
数字 17 的个数：14
数字 18 的个数：41
数字 19 的个数：80
数字 20 的个数：23
数字 21 的个数：36
数字 22 的个数：15
数字 23 的个数：39
数字 24 的个数：12
数字 25 的个数：26
数字 26 的个数：25
数字 27 的个数：130
数字 28 的个数：9
数字 29 的个数：41
数字 30 的个数：14
数字 31 的个数：15
数字 32 的个数：13
数字 33 的个数：111
数字 34 的个数：19
数字 35 的个数：51
数字 36 的个数：12
数字 37 的个数：12
数字 38 的个数：45
数字 39 的个数：14
数字 40 的个数：38
数字 41 的个数：62
数字 42 的个数：11
数字 43 的个数：13
数字 44 的个数：20
数字 45 的个数：35
数字 46 的个数：47
数字 47 的个数：17
数字 48 的个数：23
数字 49 的个数：12
数字 50 的个数：18
数字 51 的个数：11
数字 52 的个数：64
数字 53 的个数：30
数字 54 的个数：31
数字 55 的个数：57
数字 56 的个数：42
数字 57 的个数：26
数字 58 的个数：74
数字 59 的个数：39
数字 60 的个数：36
数字 61 的个数：29
数字 62 的个数：119
数字 63 的个数：27
数字 64 的个数：13
数字 65 的个数：48
数字 66 的个数：27
数字 67 的个数：14
数字 68 的个数：31
数字 69 的个数：37
数字 70 的个数：27
数字 71 的个数：9
数字 72 的个数：40
数字 73 的个数：39
数字 74 的个数：11
数字 75 的个数：15
数字 76 的个数：31
数字 77 的个

In [26]:
W = torch.nn.Parameter(torch.randn(32, 32))
a = torch.nn.Parameter(torch.randn(32))
muti_int=torch.randn(10,32,32)


In [29]:
a

Parameter containing:
tensor([ 0.6058, -2.1326,  0.2567,  0.0131, -0.5132,  0.9505,  0.0620,  1.8234,
         0.3953, -0.8554,  0.6275,  0.8167, -0.2307, -0.3301,  0.1686, -0.5537,
         0.2421,  0.0564,  1.0861, -0.4689, -0.1740, -1.7774, -0.7956,  1.9360,
        -0.9469, -0.6740, -0.8613,  0.6263, -0.3515, -0.2836,  0.1727,  0.3628],
       requires_grad=True)

In [ ]:
weight = torch.matmul(muti_int, self.W)#32 32 32 32
weight = F.softmax(torch.matmul(weight, self.a), dim = 0).unsqueeze(-1)#32 32 *32-->32 1
muti_int = torch.sum(muti_int * weight, dim = 1)#65000 32 32 32 1